In [1]:
# Instala o banco de dados vetorial ChromaDB
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/24/2a/5cad8436a39c45f9fb9a6a0662d6254039f90a5710cf82410b685bbb7c71/chromadb-0.4.17-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=1.9 from https://files.pythonhosted.org/packages/e2/2c/9906b7abc337b0250a5634de5396e2f3cb1b837af0616424c2225a65aa80/pydantic-2.5.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 416.8 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/48/0e/068b658a547d6090b969014146321e28dae1411da54b76d081e51a2af22b/chroma_hnswlib-0.7.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/f3/4f/0ce34195b63240b6693086496c9bab4ef23999112184399a3e88854c7674/fastapi-0.104.1-py3-none-any.whl.metadata
  Obtaining dependency informat

In [2]:
# Instala a biblioteca que possibilita a manipulação de LLMs
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 685.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 108.8 kB/s eta 0:00:0000:0100:01
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/f2/b8/b1ec82fce93064a73ba67f2bb158ec9cac4a0e8f0b6942268ec963947329/regex-2023.10.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 355.5 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency informatio

In [3]:
import pandas as pd
import networkx as nx
import chromadb
from sentence_transformers import SentenceTransformer

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#url do dataset completo: https://github.com/fergiemcdowall/reuters-21578-json/tree/master/data/full
# O campo body deve passar por um pré-processamento alterando \n por espaço

file_name = '/workspaces/DataScience_class/atividades/Avaliacao II/reuters-000.json'

import json

data = json.load(open(file_name, "r"))
df_list = list()
for item in data:
  df_list.append(item)

df = pd.DataFrame(df_list)
df

,title,body,date,topics,places,id,organisations
0,BAHIA COCOA REVIEW,Showers continued throughout the week in\nthe ...,26-FEB-1987 15:01:01.79,[cocoa],"[el-salvador, usa, uruguay]",1,NaN
1,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT,Standard Oil Co and BP North America\nInc said...,26-FEB-1987 15:02:20.00,NaN,[usa],2,NaN
2,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN,Texas Commerce Bancshares Inc's Texas\nCommerc...,26-FEB-1987 15:03:27.51,NaN,[usa],3,NaN
3,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER,BankAmerica Corp is not under\npressure to act...,26-FEB-1987 15:07:13.72,NaN,"[usa, brazil]",4,NaN
4,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The U.S. Agriculture Department\nreported the ...,26-FEB-1987 15:10:44.60,"[grain, wheat, corn, barley, oat, sorghum]",[usa],5,NaN
...,...,...,...,...,...,...,...
995,ASHTON-TATE <TATE> TO OFFER COMMON SHARES,Ashton-Tate said it intends to\nfile for an of...,3-MAR-1987 09:15:05.40,NaN,[usa],996,NaN
996,KEYCORP <KEY> REGISTERS SUBORDINATED NOTES,KeyCorp said it has filed with the\nSecurities...,3-MAR-1987 09:15:33.28,NaN,[usa],997,NaN
997,<NATIONAL SEA PRODUCTS LTD> 4TH QTR NET,"Oper shr 43 cts vs 21 cts\n Oper net 6,846,...",3-MAR-1987 09:15:47.27,[earn],[usa],998,NaN
998,U.K. MONEY MARKET SHORTAGE FORECAST REVISED DOWN,The Bank of England said it had revised\nits f...,3-MAR-1987 09:16:02.85,"[money-fx, interest]",[uk],999,NaN


In [5]:
# Cria uma instância do SBERT considerando o LLM all-MiniLM-L6-v2
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
doc = 'Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century.'


In [7]:
# Gera o embedding para o conteúdo de doc
embedding = embedding_model.encode(doc)
embedding

array([-2.71100574e-03,  5.03977500e-02, -2.57082786e-02,  7.46537745e-02,
       -1.37390485e-02,  6.82744384e-02,  1.18316533e-02,  2.83373706e-02,
       -1.81542058e-02, -5.48329763e-02,  6.93788659e-03, -5.54123940e-03,
       -1.17441177e-01,  6.24496229e-02, -6.32977188e-02,  7.61467740e-02,
       -4.69376333e-02,  5.57096116e-02, -9.90343988e-02, -1.09821200e-01,
       -1.30146474e-01, -2.89768334e-02, -3.15309013e-03, -9.02231485e-02,
        4.11917455e-02,  4.22021933e-02,  5.57087511e-02, -3.74031216e-02,
        3.14993039e-02,  6.18938692e-02,  2.11922545e-02, -5.76338731e-02,
        1.32149998e-02, -2.64004190e-02, -1.85544081e-02,  8.23569018e-03,
        3.03386524e-02,  1.55530870e-03,  5.77915236e-02, -1.88199114e-02,
       -3.08605488e-02,  1.38776973e-02, -4.07127626e-02,  4.56741080e-02,
        3.98645326e-02, -2.01962944e-02,  8.56906697e-02, -1.47546604e-02,
       -5.87980337e-02,  2.70214174e-02, -3.65695953e-02,  9.70428716e-03,
        2.91014705e-02, -

In [8]:
# Cria uma instância do ChromaDB
client = chromadb.Client()
collection = client.create_collection("collection")